# Step 3: Evaluate processed interferograms

This notebook:
1. Loads the S3 file system
2. Does a quick overview of success/fail for processing, and grabs error logs
3. Loops through succesful interferograms and records evaluation info for each

## 3.1 Import Python packages

In [ ]:
import s3fs
import geoviews as gv
import hvplot.xarray
import xarray as xr
import numpy as np
import geopandas as gpd
import hvplot.pandas

## 3.2 Load the S3 file system
This step loads the S3 bucket as thought it were a file system on the local computer, using credentials in the Jupyter Hub (I think?)

You'll need the following from Step 2 (or the summary TXT file):
* AWS bucket name("dirname")

In [ ]:
dirname = 'A76-1month-All-TEST' #name of the folder on AWS where your results/processing files will be
# set full path to buckets on S3
proc_bucket = 's3://dinosar/processing/uturuncu/' + dirname
res_bucket = 's3://dinosar/results/uturuncu/' + dirname
# ANY OTHER PARAMETERS NEEDED FOR GRABBING THE LOG FILES, ETC

In [ ]:
fs = s3fs.S3FileSystem() #load the file system
proc_folders = fs.ls(proc_bucket) 
res_folders = fs.ls(res_bucket) 

## 3.3 Check which interferograms succeeded or failed

### 3.3.1 Check to see which interferograms succeeded:

In [ ]:
#Compare number of processing to results directories
num_pass = len(res_folders)
num_pass

In [ ]:
num_fail = len(proc_folders)-num_pass
num_fail #this seems to be one off?

### 3.3.2 Grab logs/error outputs for failed interferograms

In [ ]:
# SOME SORT OF NIFTY CODE THAT RECREATES ELIZABETH'S AWESOME TABLE IN A PANDAS ARRAY THING

### 3.3.3 Print a report

In [ ]:
# print a basic text file with only the error text?  Or just which failed and which succeeded, with links.

## 3.4 Load and evaluate successful interferograms

Loop through successful interferograms, view and plot them, and record notes in a file for later reference.

(Eventually this should be a loop of some sort, but for now I'm putting in the code to just view 1 interferogram.)

In [ ]:
# print out successful ints
res_folders

In [ ]:
# load an unwrapped, geocoded interferogram

#Set the int to evaluate (hopefully this can be replaced with a nifty loop?)
evalint = 'int-20170602-20170509'


s3Path = res_bucket + '/' + evalint + '/filt_topophase.unw.geo' #set the path
da = xr.open_rasterio(s3Path)#load the interferogram
da['band'] = ['amplitude','phase'] # set amplitude and phase as bands
ds = da.to_dataset(dim='band')

amp=ds['amplitude']
phs=ds['phase']

In [ ]:
# Deal with nans in the data.  Can be slow.
amp = amp.where(amp != 0)

In [ ]:
phs = phs.where(phs != 0)

In [ ]:
# plot the amplitude
img = amp.hvplot.image(geo=True, rasterize=True, logz=True, cmap='gray', frame_width=500)
utur = gv.Points([(-67.18, -22.27)]).opts(line_color='r', fill_color=None, marker='^', size=10)
img * utur

In [ ]:
# plot the unwrapped phase
img = phs.hvplot.image(geo=True, rasterize=True, cmap='bwr', frame_width=500)
img * utur

In [ ]:
# Enter evaluation info on the int in some standardized format.  Suggested:
# Unwrapping error present? (Y/N)
# Unwrapping error present at target? (Y/N)
# Burst alignment error present? (Y/N)
# Burst alignment error present at target? (Y/N)
# Signal of interest (e.g., an earthquake or something) (Y/N)
# Comments (Free-form text entry)